In [9]:
############## ZZ[sqrt(d)] ###################

fundamental_units={} # this will ensure we only ever try to compute fundamental units for each ring of integers once, and then it will remember them

def solvePell(N, val): #solves abs(x^2-N*y^2)=val
    N0 = sqrt(N)  # Start with the initial value of N0
    a = floor(N0)  # The floor of N
    p = (1, a)  # p starts as (1, a)
    q = (0, 1)  # q starts as (0, 1)
    if val==1:
        while 1:
            if abs(p[1]^2-N*q[1]^2)==1:
                return p[1],q[1]
            N0 = 1 / (N0 - a)  # Update N0
            a = floor(N0)  # Update a
            p = (p[1],p[1] * a + p[0])  # Update p
            q = (q[1],q[1] * a + q[0])  # Update q
    if val==4:
        ul=oo # initialise ul to infinity
        while 1:
            if 2*p[1]+2*q[1]*sqrt(N) > ul:
                return poss
            if abs(p[1]^2-N*q[1]^2)==4:
                return p[1],q[1]
            if abs(p[1]^2-N*q[1]^2)==1 and ul==oo:
                poss=(2*p[1],2*q[1])
                ul = 2*p[1]+2*q[1]*sqrt(N)
            N0 = 1 / (N0 - a)  # Update N0
            a = floor(N0)  # Update a
            p = (p[1],p[1] * a + p[0])  # Update p
            q = (q[1],q[1] * a + q[0])  # Update q

class Zsqrtd: #creating a class for the ring of integers for Q(sqrt(d))
    def __init__(self,integer,root=0,d=-1): #initialise an element
        self.integer=integer
        self.root=root #if d is 1 mod 4 then this is (1+sqrt(d))/2
        #if d not squarefree raise an error
        for f in range(2,int(abs(d)**0.5)+1):
            if d%f**2==0:
                raise ValueError("d must be squarefree")
        self.d=d
        #now lets check if we already know a fundamental unit!
        if d>0:
            if d not in fundamental_units.keys():
                fundamental_units[d]=0#so now d is in fundamental_units.keys()
                if d%4==1:
                    sol=solvePell(d,4)
                    #so sol[0]/2+sol[1]/2*sqrt(d) is our fundamental unit, which is written as
                    #sol[0]/2-sol[1]/2 + sol[1] (1+sqrt(d))/2
                    fundamental_units[d]=Zsqrtd(sol[0]/2-sol[1]/2,sol[1],d)
                else:
                    sol=solvePell(d,1)
                    fundamental_units[d]=Zsqrtd(sol[0],sol[1],d)
    def __repr__(self): # how to represent a number in Q(sqrt(d))
        if self.root >=0:
            if self.d % 4 != 1:
                return str(self.integer) +"+" + str(self.root)+ "sqrt("+ str(self.d) +")"
            else:
                return str(self.integer) + "+" + str(self.root)+ "(1+sqrt("+ str(self.d) +"))/2"
        else: 
            if self.d % 4 != 1:
                return str(self.integer)  + str(self.root)+ "sqrt("+ str(self.d) + ")"
            else:
                return str(self.integer)  + str(self.root)+ "(1+sqrt("+ str(self.d) +"))/2"
        
    def __add__(self,other): # defining addition
        assert self.d==other.d #gives error if d's arent the same 
        return Zsqrtd(self.integer+other.integer,self.root+other.root,self.d)
    def __sub__(self,other): #defining subtraction
        assert self.d==other.d
        return Zsqrtd(self.integer-other.integer,self.root-other.root,self.d)
    
    def __mul__(self,other): #defining multiplication
        assert self.d==other.d
        if self.d % 4 != 1:
            return Zsqrtd(self.integer*other.integer+self.d*self.root*other.root, self.integer*other.root+self.root*other.integer,self.d)
        return Zsqrtd(self.integer*other.integer+self.root*other.root*(self.d-1)//4,self.root*other.integer+other.root*self.integer+self.root*other.root,self.d)
    def __pow__(self,other):
        out=Zsqrtd(self.integer,self.root,self.d)
        for i in range(other):
            out = out * self
        return out
    def conj(self):
        if self.d % 4 != 1:
            return Zsqrtd(self.integer, -self.root,self.d)
        return Zsqrtd(self.integer+self.root, -self.root,self.d)
    def __floordiv__(self,other): # defining floor division so we can take mods, we dont need truediv
        assert self.d==other.d
        denom = (other.conj() * other).integer
        if denom == 0:
            print(other.conj() * other)
            print(other)
            assert 0
        numer = other.conj() * self
        return Zsqrtd(numer.integer//denom,numer.root//denom,self.d)
    
    def __mod__(self,other) : #defining mod := the remainder after floor division
        assert self.d==other.d
        return self-((self//other)*other)
    def __eq__(self,other):
        assert self.d==other.d
        return (self.integer==other.integer) and (self.root==other.root)
    def norm(self):
        return (self*self.conj()).integer
    def withoutunits(self):
        if not self.integer==self.root==0:
            if self.d==-1:
                I=Zsqrtd(0,1,-1)
                while not (self.integer>0 and self.root>=0):
                    self = self * I
            elif self.d==-3:
                w=Zsqrtd(0,1,-3) # this generates the group of units
                while not (self.integer*2+self.root>0 and self.root>=0 and (self * w * w).root>0):
                    self = self * w
            elif self.d<0:
                #units are only \pm 1 so its easy!
                if self.root < 0:
                    self = self * Zsqrtd(-1,0,self.d)
                elif self.root == 0:
                    if self.integer < 0:
                        self = self * Zsqrtd(-1,0,self.d)
            else: #self.d>0
                if self.d%4==1:
                    if self.integer+self.root/2+self.root/2*sqrt(self.d)<0:
                        self = self * Zsqrtd(-1,0,self.d)
                    while self.integer+self.root*(1+sqrt(self.d))/2 < 1:
                        self = self * fundamental_units[self.d]
                    while (self // fundamental_units[self.d]).integer+(self // fundamental_units[self.d]).root*(1+sqrt(self.d))/2 > 1:
                        self = self // fundamental_units[self.d]
                else:
                    if self.integer+self.root*sqrt(self.d) < 0:
                        self = self * Zsqrtd(-1,0,self.d)
                    while self.integer+self.root*sqrt(self.d) < 1:
                        self = self * fundamental_units[self.d]
                    while (self // fundamental_units[self.d]).integer+(self // fundamental_units[self.d]).root*sqrt(self.d) > 1:
                        self = self // fundamental_units[self.d]
        return self


In [10]:
def trialcommfact(a,b):#a,b are in Zsqrtd and we aim to find a common factor instead of a gcd
    assert a.d==b.d
    for i in range(max(abs(a.integer),abs(a.root),abs(b.integer),abs(b.root))+1):#is this range correct when d>0? NO
        for j in range(-max(abs(a.integer),abs(a.root),abs(b.integer),abs(b.root)),max(abs(a.integer),abs(a.root),abs(b.integer),abs(b.root))+1):
            if i!=0 or j!=0:
                f=Zsqrtd(i,j,d=a.d)
                if a%f==Zsqrtd(0,d=a.d) and b%f== Zsqrtd(0,d=a.d) and abs(f.norm())!=1:
                    return f
    return Zsqrtd(1,d=a.d)

def commfact(a,b):
    assert a.d==b.d
    while not b == Zsqrtd(0,0,b.d):
        beforenm=b.norm()
        a,b = b,(a%b).withoutunits()
        if b.norm()>beforenm and a.d not in [-11, -7, -3, -2, -1, 2, 3, 5, 6, 7, 11, 13, 17, 19, 21, 29, 33, 37, 41, 57, 73]:#the point is that if we are in a euclidean domain, we shouldn't need to do trial division. The list of values of d is from wikipedia https://en.wikipedia.org/wiki/Euclidean_domain
            break
    if b == Zsqrtd(0,0,b.d):
        return a
    return trialcommfact(a,b)

In [11]:
####################### INTEGER FACTORING METHODS ####################################

def modexp(a,m,n):#computes a**m%n by repeated squaring
    if m==1:
        return a%n
    elif m%2 == 0:
        return modexp((a*a)%n,m//2,n)
    else:
        return (a*modexp((a*a)%n,(m-1)//2,n))%n
    
def lcmB(B):#computes lcm of 1,2,3,...,B
    product = 1
    for p in primes(B):
        product *= p^(int(log(B ,p))) 
    return product

def modinv(a,n):#computes a^-1 mod n using extended euclidean algorithm
    a%=n
    r=(0,1)
    #s=(1,0)
    N=(n,a)
    #at all times, we should have N[0]=r[0]*a+s[0]*n
    while N[1]!=0:
        d=N[0]//N[1] #
        r=(r[1],r[0]-d*r[1])
        #s=(s[1],s[0]-d*s[1])
        N=(N[1],N[0]%N[1])
    assert N[0]==1# gcd(a,n)!=1
    return r[0]%n

def pollard(N,B=15,m=0):#implements pollards p-1 algorithm
    if not m: #this way we don't compute lcmB(B) many times if we run the algorithm many times
        m = lcmB(B)
    for i in range(100):
        a=randint(2,N-2)
        am_1 = modexp(a,m,N) - 1
        factor = gcd(am_1,N)
        if 1 < factor < N:
            return factor,N//factor
    return -1
               
def ECadd(a,N,P,Q): # add points P & Q over the elliptic curve y^2 = x^3 + ax + 1
    if P == "inf": #case 1: P is infinity
        return Q
    if Q == "inf": #case 2: Q is infinity
        return P
    if P[0] == Q[0] and (P[1] + Q[1])%N==0: #case 3: points on same x line
        return "inf"
    else: #compute gradient, use this to find new (x,y)
        if P==Q:
            grad = ((3*P[0]**2 + a)*(modinv(2*P[1],N)))%N
        else:
            grad = ((P[1]-Q[1])*(modinv(P[0]-Q[0],N)))%N
        x = (grad**2 - P[0]- Q[0])%N #x co-ord of output (R in wstein ENT alg 6.2.1)
        y = (-grad*x-P[1]+grad*P[0])%N
        return (x,y)
           
def lenstraTrial(a,N,m):#does lenstras algorithm on the EC y^2=x^3+ax+1
    P=(0,1)
    mP=(0,1)# use this to compute mP
    m=bin(m)[2:]#this is m in binary as a string of 0s and 1s
    #now calculate m*P
    for bit in m[1:]:#looks at bits but ignores the first to do repeated doubling
        try:
            mP=ECadd(a,N,mP,mP)
        except:#then we have found a nonzero gcd in denom, which is 2P[1] in ECadd
            return gcd(N,mP[1])#y1 is a non-unit in Z/NZ so this is a non-trivial factor
        if bit=='1':
            try:
                mP=ECadd(a,N,mP,P)
            except: #we found a non 0 gcd in denom, which is P[0]-Q[0]=mP[0]
                return gcd(N,mP[0])
    return N#failed to find a non-trivial factor

def lenstra(N,B=15,numTrials=10,m=0):#trying different elliptic curves until one works
    if N%2==0:
        return 2,N//2
    if not m:#if we have pre-computed an m, don't compute it again!
        m=lcmB(B)
    for i in range(numTrials):
        a=randint(1,N-1)
        g=lenstraTrial(a,N,m)
        if 1<g<N:
            return g,N//g
    return -1#failed to find a factor

In [29]:
####################################################### NORM METHOD ###########################################################
######## note that this method to factorise the norm doesnt nessecarily work for all rings of integers but it works well for Gaussian Integers ########

def lenstraNorm(N,B=10,m=0):#implements lenstra algorithm for ZZ[i] using a different method
    if not m:
        m = lcmB(B)
    normN = N.norm() #use the norm of our gaussian integer 
    f = lenstra(normN,m=m)
    if f==-1:
        return -1
    f=f[0]
    factor = commfact(N,Zsqrtd(f,0,N.d))
    return factor,N//factor


In [13]:
def pollardZsqrtd(N,B=15,m=0):#implements pollards p-1 algorithm where N is a Zsqrtd object
    if not m: #this way we don't compute lcmB(B) many times if we run the algorithm many times
        m = lcmB(B)
    for i in range(300):
        a=Zsqrtd(randint(2,1000),randint(2,1000),N.d)
        am_1 = modexp(a,m,N) - Zsqrtd(1,0,N.d)
        factor = commfact(am_1,N)
        if 1 < abs(factor.norm()) < abs(N.norm()):
            return factor,N//factor
    return -1

########################## DIXONS'S RANDOM SQUARES#########################################
def Bsmoothnesssqrtd(N,primesuptoB): #this is so we can find suitible k's that are B smooth
    output = []
    zero=Zsqrtd(0,0,N.d)
    for p in primesuptoB:
        count = 0
        while N%p==zero:
            N=N//p
            count +=1
        output.append(count) # adds the power to the list
    if abs(N.norm())==1:
        return output 
    return False #fails if not B smooth

def Zsqrtdprimes(B,d): # finds a list of primes in Zsqrtd
    assert d<0 # else this method won't work in its current format
    start=[]
    out=[]
    for i in range(-floor(sqrt(B)),floor(sqrt(B))+1):#generate a list of all the numbers
        for j in range(-floor(sqrt((i^2-B)/d)),floor(sqrt((i^2-B)/d))+1):
            x=Zsqrtd(i,j,d).withoutunits()
            if B>=x.norm()>1:
                if x not in start:
                    start.append(x)
    #sort the list by norm size
    start = sorted(start, key=lambda x: x.norm())
    #now we can do sieve of eratosthenes same as before!
    zero=Zsqrtd(0,0,d)
    while start != []:
        k=start.pop(0)
        out.append(k)
        i=0
        while i<len(start):
            if start[i]%k==zero:
                start.pop(i)
            else:
                i+=1
    return out
        

def dixonsZsqrtd(N,B=100):#this works! however, when working over a non-euclidean domain, this is rather slow...
    P=Zsqrtdprimes(B,N.d) #get a list of the primes of norm up to B
    goodA=[] #initialise a list of a's that are squares
    goodK=[]
    J = matrix(ZZ)
    lb=int(sqrt(abs(N.norm()))) #bounds for our random a's
    ub=abs(N.norm())-lb
    zero=Zsqrtd(0,0,N.d)
    for _ in range(100):
        a=Zsqrtd(randint(lb,ub),randint(lb,ub),N.d)
        k= (a*a)%N
        if k==zero: #need to deal with this case separately otherwise Bsmoothness will go on forever
            f=commfact(a,N)
            return f,N//f
        kFactors = Bsmoothnesssqrtd(k,P)
        if kFactors:
            J=matrix(ZZ,J.rows()+[kFactors]) #adds the row kFactors to the matrix J
            goodA.append(a)
            goodK.append(k)
            x=matrix(GF(2),J).left_kernel().random_element() #takes a random element in the null space of matrix J mod 2
            A,B=Zsqrtd(1,0,N.d),Zsqrtd(1,0,N.d) 
            for i in range(len(x)): # A is the product of good A's 
                if x[i]==1:
                    A*=goodA[i]
            y=1/2*matrix(ZZ,x)*J #This will give the prime factorisation of B
            for i in range(len(P)): 
                B*=P[i]^y[0,i] #calculates the integer B so now we have A^2=B^2 mod N
            g=commfact(A-B,N)
            if 1<abs(g.norm())<abs(N.norm()):
                return g,N//g
    return -1

In [30]:
##################### Factorise a random nonprime gaussian integer using lenstra norm ###################################

a=Zsqrtd(randint(2,100),randint(2,100),-1)*Zsqrtd(randint(2,100),randint(2,100),-1)
print(a)
print(lenstraNorm(a))

-1013+10049sqrt(-1)
(1+1sqrt(-1), 4518+5531sqrt(-1))


In [31]:
################# Factorise a random nonprime gaussian integer using pollard ###################

a=Zsqrtd(randint(2,100),randint(2,100),-1)*Zsqrtd(randint(2,100),randint(2,100),-1)
print(a)
print(pollardZsqrtd(a))

-314+5442sqrt(-1)
(43+36sqrt(-1), 58+78sqrt(-1))


In [32]:
################# Factorise a random ZZ[sqrt(-2)] using pollard ###################

a=Zsqrtd(randint(2,50),randint(2,50),-2)*Zsqrtd(randint(2,50),randint(2,50),-2)
print(a)
print(pollardZsqrtd(a)) #sometimes this runs forever 

-432+756sqrt(-2)
(11+14sqrt(-2), 32+28sqrt(-2))


In [36]:
############### Factorise a random nonprime gaussian integer using dixons squares #####################

a=Zsqrtd(randint(2,100),randint(2,100),-1)*Zsqrtd(randint(2,100),randint(2,100),-1)
print(a)
print(dixonsZsqrtd(a))

-1249+903sqrt(-1)


-1


In [37]:
############################## Factorise the semi-prime 35327*34537 by factorising over the Gaussian integers ##############################

print(pollardZsqrtd(Zsqrtd(35327*34537,0,-1),100)) #This only works some of the time
#We found 101+156i as a factor of 35327*34537, so we will now take the norm of this factor to hopefully get a factor in the integers
print(Zsqrtd(101,156,-1).norm()) 

(101+156sqrt(-1), 3568027-5511012sqrt(-1))
34537


In [13]:
############################## Comparison of factorising 35327*34537 by factorising over the Gaussian integers and over the integers normally ##############################

def pollardGaussianVIntegers(N):
    goodI=0
    goodG=0
    badI=0
    badG=0
    for _ in range(10):
        integer = pollard(N,100)
        gaussian = pollardZsqrtd(Zsqrtd(N,0,-1),100)
        if integer==-1:
            badI += 1
        else:
            goodI += 1
            
        if gaussian==-1:
            badG += 1
        else:
            goodG += 1
    print(f"pollard's over the integers failed {badI} times")
    print(f"pollard's over the gaussian integers failed {badG} times")

pollardGaussianVIntegers(35327*34537)    

pollard's over the integers failed 10 times
pollard's over the gaussian integers failed 8 times


In [38]:
######################### LOG TIME GRAPH FOR POLLARD & LENSTRA NORM & DIXONS ##########################################
from time import time
from matplotlib import pyplot as plt
#Taking B=15 for both pollard and lenstra, and B=100 for dixon, as these values work best for each, we want to compare time to the best performance
def timegraph(reps=500,B=15):
    m=lcmB(B)
    tpollard=[]
    pollardfails=0
    tlenstra=[]
    lenstrafails=0
    tdixon=[]
    dixonfails=0
    for i in range(reps):
        n=Zsqrtd(randint(2,100),randint(2,100),-1)*Zsqrtd(randint(2,100),randint(2,100),-1)
            
        start=time()
        for j in range(1):
            f=pollardZsqrtd(n,m=m)
            if f==-1:
                pollardfails+=1
        tpollard.append(log(time()-start))
        
        start=time()
        for j in range(1):
            f=lenstraNorm(n,m=m)
            if f==-1:
                lenstrafails+=1
        tlenstra.append(log(time()-start))
        
        start=time()
        for j in range(1):
            f=dixonsZsqrtd(n,B=100)
            if f==-1:
                dixonfails+=1
        tdixon.append(log(time()-start))
        
    plt.hist(tpollard,color='r',alpha=0.5,bins=30)
    plt.hist(tlenstra,color='b',alpha=0.5,bins=30)
    plt.hist(tdixon,color='g',alpha=0.5,bins=30)
    plt.legend(['Pollard','Lenstra Norm','Dixon'])
    plt.xlabel('log of time')
    plt.ylabel('frequency')
    plt.show()
    print(f"pollard failed {pollardfails} times")
    print(f"lenstra failed {lenstrafails} times")
    print(f"dixon failed {dixonfails} times")

In [39]:
#timegraph()

In [40]:
def pollardGraphGaussian(N):
    B=list(range(3,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=pollardZsqrtd(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

a=Zsqrtd(76,61,-1)*Zsqrtd(117,50,-1)    
#pollardGraphGaussian(a)

In [0]:
def lenstraNormGraphGaussian(N):
    B=list(range(3,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=lenstraNorm(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

a=Zsqrtd(76,61,-1)*Zsqrtd(117,50,-1)    
#lenstraNormGraphGaussian(a)

In [0]:
def dixonsGraphGaussian(N):
    B=list(range(50,400,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=dixonsZsqrtd(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

a=Zsqrtd(76,61,-1)*Zsqrtd(117,50,-1)    
#dixonsGraphGaussian(a)